In [5]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from pathlib import Path
from ollama import chat
from ollama import ChatResponse


path = 'data'
root_folder = Path(path)
root_folder.mkdir(parents=True, exist_ok=True)


In [35]:
path = 'data'
 

df_searchterms = pd.read_csv(path+'/google_scholar_search_base_article_detail_txt.csv',sep =';')
df_searchterms['folder_score'] = path +'/score/'+ df_searchterms['folder']
df_searchterms['file_score'] = df_searchterms['folder_score']+'/'+df_searchterms['article_file'].str.replace('.html','.txt').str.replace('.pdf','.txt')


In [36]:
files = list((root_folder/'score' ).rglob("*"))  
files = [f for f in files if f.is_file() and f.suffix in (".txt")]
lst_name_files = [file.name for file in files]
s = df_searchterms['article_file'].str.replace('.html','.txt').str.replace('.pdf','.txt') 
df_searchterms = df_searchterms[~s.isin(lst_name_files)].copy()

In [37]:
records = df_searchterms.to_dict(orient = 'records')

In [38]:
f= open("prompt.xml", "r", encoding="utf-8") 
prompt = f.read()
f.close()


In [39]:
for record in tqdm(records):


    full_path_folder = Path(record['folder_score'])
    full_path_folder.mkdir(parents=True, exist_ok=True)

    full_path_txt = Path(record['path_txt_article'])
    full_path_score = Path(record['file_score'])

    if not(full_path_txt.exists() and full_path_txt.is_file()):  
        continue

    if full_path_score.exists() and full_path_score.is_file():  
        continue

    f= open(record['path_txt_article'], "r", encoding="utf-8") 
    document = f.read()
    f.close()

    
    response: ChatResponse = chat(model= 'qwen3:30b-a3b', messages=[
    {
        'role': 'user',
        'content': prompt.format(document = document),
    },
    ])

    content = response['message']['content']
    with open(record['file_score'], "w", encoding="utf-8") as f:
        f.write(content)

    break



  0%|          | 0/1908 [00:00<?, ?it/s]